In [1]:
!pip install -q tensorflow
!pip install -q keras


## Imports libs
import os
from tensorflow import keras

In [2]:
 !pip install -q tensorflow-model-optimization

In [4]:
import tempfile
import os

import tensorflow as tf

from tensorflow import keras

In [3]:
import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot

import tempfile

In [7]:

import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import SVHN
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

In [6]:
cifar=tf.keras.datasets.cifar10.load_data()

In [8]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train / 255.0
x_test  = x_test  / 255.0 

assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [99]:
model = tf.keras.models.Sequential([
             tf.keras.layers.Conv2D(32,
                                 kernel_size=(3,3),
                                 strides=(1,1),
                                 padding="same",
                                 activation="relu",
                                 input_shape=(32,32,3)),
             tf.keras.layers.BatchNormalization(),
             tf.keras.layers.Conv2D(64,
                                 kernel_size=(3,3),
                                 strides=(1,1),
                                 padding="same",
                                 activation="relu"),
             tf.keras.layers.BatchNormalization(),
             tf.keras.layers.Conv2D(128,
                                 kernel_size=(3,3),
                                 strides=(1,1),
                                 padding="same",
                                 activation="relu"),
             tf.keras.layers.BatchNormalization(),
             tf.keras.layers.MaxPooling2D(2,2),
             tf.keras.layers.Conv2D(256,
                                 kernel_size=(3,3),
                                 strides=(1,1),
                                 padding="same",
                                 activation="relu"),
             tf.keras.layers.BatchNormalization(),
             tf.keras.layers.MaxPooling2D(2,2),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(512, activation="relu"),
             tf.keras.layers.BatchNormalization(),
             tf.keras.layers.Dropout(0.5),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(1028, activation="relu"),
             tf.keras.layers.BatchNormalization(),
             tf.keras.layers.Dropout(0.5),
             tf.keras.layers.Dense(10, activation="softmax")
       ])


In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(32, 32,3)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [100]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [101]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [106]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1563/1563 [==============================] - 403s 258ms/step - loss: 1.0890 - accuracy: 0.6282
Epoch 2/5
1563/1563 [==============================] - 391s 250ms/step - loss: 0.8526 - accuracy: 0.7050
Epoch 3/5
1563/1563 [==============================] - 384s 246ms/step - loss: 0.6927 - accuracy: 0.7627
Epoch 4/5
1563/1563 [==============================] - 368s 235ms/step - loss: 0.5787 - accuracy: 0.8011
Epoch 5/5
1563/1563 [==============================] - 367s 235ms/step - loss: 0.4563 - accuracy: 0.8432


In [107]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 15s - loss: 0.6396 - accuracy: 0.7966


[0.6395964026451111, 0.7965999841690063]

In [104]:
base_model=model

quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)
quant_aware_model.summary()

RuntimeError: Layer batch_normalization_7:<class 'keras.layers.normalization.batch_normalization.BatchNormalization'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.

In [28]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

In [29]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class FixedRangeQuantizer(tfmot.quantization.keras.quantizers.Quantizer):
  """Quantizer which forces outputs to be between -1 and 1."""

  def build(self, tensor_shape, name, layer):
    # Not needed. No new TensorFlow variables needed.
    return {}

  def __call__(self, inputs, training, weights, **kwargs):
    return tf.keras.backend.clip(inputs, -1.0, 1.0)

  def get_config(self):
    # Not needed. No __init__ parameters to serialize.
    return {}


class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    # Configure weights to quantize with 4-bit instead of 8-bits.
    def get_weights_and_quantizers(self, layer):
      # Use custom algorithm defined in `FixedRangeQuantizer` instead of default Quantizer.
      return [(layer.kernel, FixedRangeQuantizer())]

In [94]:
model = quantize_annotate_model(tf.keras.Sequential([
   
    keras.layers.InputLayer( input_shape=(32, 32,3)),
    #keras.layers.Reshape(target_shape=(32, 32, 3)),
      # Pass in modified `QuantizeConfig` to modify this `Dense` layer.
    quantize_annotate_layer(tf.keras.layers.Dense(30,), ModifiedDenseQuantizeConfig()),
    
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3), activation="relu"),
    tf.keras.layers.Conv2D(128,kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256,kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dense(1028, activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax")
    
    
       
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_11 (QuantizeL (None, 32, 32, 3)         3         
_________________________________________________________________
quant_dense_47 (QuantizeWrap (None, 32, 32, 30)        123       
_________________________________________________________________
quant_conv2d_75 (QuantizeWra (None, 30, 30, 12)        3279      
_________________________________________________________________
quant_max_pooling2d_55 (Quan (None, 15, 15, 12)        1         
_________________________________________________________________
quant_conv2d_76 (QuantizeWra (None, 13, 13, 64)        7107      
_________________________________________________________________
quant_conv2d_77 (QuantizeWra (None, 11, 11, 128)       74115     
_________________________________________________________________
quant_max_pooling2d_56 (Quan (None, 5, 5, 128)       

In [95]:
# `quantize_model` requires a recompile.
quant_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

quant_aware_model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_11 (QuantizeL (None, 32, 32, 3)         3         
_________________________________________________________________
quant_dense_47 (QuantizeWrap (None, 32, 32, 30)        123       
_________________________________________________________________
quant_conv2d_75 (QuantizeWra (None, 30, 30, 12)        3279      
_________________________________________________________________
quant_max_pooling2d_55 (Quan (None, 15, 15, 12)        1         
_________________________________________________________________
quant_conv2d_76 (QuantizeWra (None, 13, 13, 64)        7107      
_________________________________________________________________
quant_conv2d_77 (QuantizeWra (None, 11, 11, 128)       74115     
_________________________________________________________________
quant_max_pooling2d_56 (Quan (None, 5, 5, 128)       

In [97]:
quant_aware_model.fit(x_train, y_train,  batch_size=128, epochs=5)

Epoch 1/5
391/391 [==============================] - 64s 165ms/step - loss: 1.4384 - accuracy: 0.4743
Epoch 2/5
391/391 [==============================] - 63s 160ms/step - loss: 1.2055 - accuracy: 0.5700
Epoch 3/5
391/391 [==============================] - 63s 162ms/step - loss: 1.0695 - accuracy: 0.6239
Epoch 4/5
391/391 [==============================] - 60s 153ms/step - loss: 0.9527 - accuracy: 0.6666
Epoch 5/5
391/391 [==============================] - 66s 168ms/step - loss: 0.8509 - accuracy: 0.7020


In [98]:
quant_aware_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 3s - loss: 1.0360 - accuracy: 0.6530


[1.036031723022461, 0.652999997138977]

In [86]:
 model = quantize_annotate_model(tf.keras.Sequential([
   
             keras.layers.InputLayer( input_shape=(32, 32,3)),
    #keras.layers.Reshape(target_shape=(32, 32, 3)),
      # Pass in modified `QuantizeConfig` to modify this `Dense` layer.
             quantize_annotate_layer(tf.keras.layers.Dense(30,), ModifiedDenseQuantizeConfig()),
             tf.keras.layers.Conv2D(256,kernel_size=(3,3),activation="relu"),
             tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation="relu"),
             tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation="relu"),
             tf.keras.layers.MaxPooling2D(2,2),
             tf.keras.layers.Conv2D(256,kernel_size=(3,3),activation="relu"),
             tf.keras.layers.MaxPooling2D(2,2),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(512, activation="relu"),        
             tf.keras.layers.Dropout(0.5),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(1028, activation="relu"),            
             tf.keras.layers.Dropout(0.5),
             tf.keras.layers.Dense(10, activation="softmax")
     ]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_10 (QuantizeL (None, 32, 32, 3)         3         
_________________________________________________________________
quant_dense_43 (QuantizeWrap (None, 32, 32, 30)        123       
_________________________________________________________________
quant_conv2d_71 (QuantizeWra (None, 30, 30, 256)       69891     
_________________________________________________________________
quant_conv2d_72 (QuantizeWra (None, 28, 28, 64)        147651    
_________________________________________________________________
quant_conv2d_73 (QuantizeWra (None, 26, 26, 128)       74115     
_________________________________________________________________
quant_max_pooling2d_53 (Quan (None, 13, 13, 128)       1         
_________________________________________________________________
quant_conv2d_74 (QuantizeWra (None, 11, 11, 256)     

In [87]:
# `quantize_model` requires a recompile.
quant_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

quant_aware_model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_10 (QuantizeL (None, 32, 32, 3)         3         
_________________________________________________________________
quant_dense_43 (QuantizeWrap (None, 32, 32, 30)        123       
_________________________________________________________________
quant_conv2d_71 (QuantizeWra (None, 30, 30, 256)       69891     
_________________________________________________________________
quant_conv2d_72 (QuantizeWra (None, 28, 28, 64)        147651    
_________________________________________________________________
quant_conv2d_73 (QuantizeWra (None, 26, 26, 128)       74115     
_________________________________________________________________
quant_max_pooling2d_53 (Quan (None, 13, 13, 128)       1         
_________________________________________________________________
quant_conv2d_74 (QuantizeWra (None, 11, 11, 256)     

In [92]:
quant_aware_model.fit(x_train, y_train,  batch_size=128, epochs=5)

Epoch 1/5
391/391 [==============================] - 456s 1s/step - loss: 0.6658 - accuracy: 0.7675
Epoch 2/5
391/391 [==============================] - 444s 1s/step - loss: 0.6009 - accuracy: 0.7886
Epoch 3/5
391/391 [==============================] - 439s 1s/step - loss: 0.5373 - accuracy: 0.8113
Epoch 4/5
391/391 [==============================] - 439s 1s/step - loss: 0.4837 - accuracy: 0.8295
Epoch 5/5
391/391 [==============================] - 456s 1s/step - loss: 0.4332 - accuracy: 0.8448


In [93]:
quant_aware_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 22s - loss: 0.7906 - accuracy: 0.7505


[0.7906006574630737, 0.7505000233650208]

In [80]:
model = quantize_annotate_model(tf.keras.Sequential([
   
    keras.layers.InputLayer( input_shape=(32, 32,3)),
    #keras.layers.Reshape(target_shape=(32, 32, 3)),
      # Pass in modified `QuantizeConfig` to modify this `Dense` layer.
    quantize_annotate_layer(tf.keras.layers.Dense(30,), ModifiedDenseQuantizeConfig()),
    
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3), activation="relu"),
    tf.keras.layers.Conv2D(128,kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256,kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax")
    
    
       
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_9 (QuantizeLa (None, 32, 32, 3)         3         
_________________________________________________________________
quant_dense_41 (QuantizeWrap (None, 32, 32, 30)        123       
_________________________________________________________________
quant_conv2d_67 (QuantizeWra (None, 30, 30, 12)        3279      
_________________________________________________________________
quant_max_pooling2d_50 (Quan (None, 15, 15, 12)        1         
_________________________________________________________________
quant_conv2d_68 (QuantizeWra (None, 13, 13, 64)        7107      
_________________________________________________________________
quant_conv2d_69 (QuantizeWra (None, 11, 11, 128)       74115     
_________________________________________________________________
quant_max_pooling2d_51 (Quan (None, 5, 5, 128)       

In [81]:
# `quantize_model` requires a recompile.
quant_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

quant_aware_model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_9 (QuantizeLa (None, 32, 32, 3)         3         
_________________________________________________________________
quant_dense_41 (QuantizeWrap (None, 32, 32, 30)        123       
_________________________________________________________________
quant_conv2d_67 (QuantizeWra (None, 30, 30, 12)        3279      
_________________________________________________________________
quant_max_pooling2d_50 (Quan (None, 15, 15, 12)        1         
_________________________________________________________________
quant_conv2d_68 (QuantizeWra (None, 13, 13, 64)        7107      
_________________________________________________________________
quant_conv2d_69 (QuantizeWra (None, 11, 11, 128)       74115     
_________________________________________________________________
quant_max_pooling2d_51 (Quan (None, 5, 5, 128)       

In [84]:
quant_aware_model.fit(x_train, y_train,  batch_size=128, epochs=10)

Epoch 1/10
391/391 [==============================] - 62s 159ms/step - loss: 0.8477 - accuracy: 0.7044
Epoch 2/10
391/391 [==============================] - 62s 159ms/step - loss: 0.7700 - accuracy: 0.7359
Epoch 3/10
391/391 [==============================] - 62s 160ms/step - loss: 0.6918 - accuracy: 0.7606
Epoch 4/10
391/391 [==============================] - 62s 160ms/step - loss: 0.6245 - accuracy: 0.7834
Epoch 5/10
391/391 [==============================] - 62s 159ms/step - loss: 0.5445 - accuracy: 0.8108
Epoch 6/10
391/391 [==============================] - 62s 159ms/step - loss: 0.4733 - accuracy: 0.8348
Epoch 7/10
391/391 [==============================] - 62s 159ms/step - loss: 0.4076 - accuracy: 0.8593
Epoch 8/10
391/391 [==============================] - 63s 160ms/step - loss: 0.3473 - accuracy: 0.8789
Epoch 9/10
391/391 [==============================] - 63s 160ms/step - loss: 0.2842 - accuracy: 0.9016
Epoch 10/10
391/391 [==============================] - 60s 154ms/step - l

In [85]:
quant_aware_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 3s - loss: 1.4938 - accuracy: 0.6686


[1.4937769174575806, 0.6686000227928162]

In [71]:
model = quantize_annotate_model(tf.keras.Sequential([
   
    keras.layers.InputLayer( input_shape=(32, 32,3)),
    #keras.layers.Reshape(target_shape=(32, 32, 3)),
      # Pass in modified `QuantizeConfig` to modify this `Dense` layer.
    quantize_annotate_layer(tf.keras.layers.Dense(30,), ModifiedDenseQuantizeConfig()),
    
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax")
    
    
       
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_8 (QuantizeLa (None, 32, 32, 3)         3         
_________________________________________________________________
quant_dense_39 (QuantizeWrap (None, 32, 32, 30)        123       
_________________________________________________________________
quant_conv2d_66 (QuantizeWra (None, 30, 30, 12)        3279      
_________________________________________________________________
quant_max_pooling2d_49 (Quan (None, 15, 15, 12)        1         
_________________________________________________________________
quant_flatten_32 (QuantizeWr (None, 2700)              1         
_________________________________________________________________
quant_dense_40 (QuantizeWrap (None, 10)                27015     
Total params: 30,422
Trainable params: 30,382
Non-trainable params: 40
________________________________________________

In [72]:
# `quantize_model` requires a recompile.
quant_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

quant_aware_model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_8 (QuantizeLa (None, 32, 32, 3)         3         
_________________________________________________________________
quant_dense_39 (QuantizeWrap (None, 32, 32, 30)        123       
_________________________________________________________________
quant_conv2d_66 (QuantizeWra (None, 30, 30, 12)        3279      
_________________________________________________________________
quant_max_pooling2d_49 (Quan (None, 15, 15, 12)        1         
_________________________________________________________________
quant_flatten_32 (QuantizeWr (None, 2700)              1         
_________________________________________________________________
quant_dense_40 (QuantizeWrap (None, 10)                27015     
Total params: 30,422
Trainable params: 30,382
Non-trainable params: 40
________________________________________________

In [78]:
quant_aware_model.fit(x_train, y_train,  batch_size=128, epochs=5)

Epoch 1/5
391/391 [==============================] - 38s 98ms/step - loss: 0.8931 - accuracy: 0.6911
Epoch 2/5
391/391 [==============================] - 39s 100ms/step - loss: 0.8866 - accuracy: 0.6912
Epoch 3/5
391/391 [==============================] - 39s 99ms/step - loss: 0.8840 - accuracy: 0.6945
Epoch 4/5
391/391 [==============================] - 37s 95ms/step - loss: 0.8767 - accuracy: 0.6950
Epoch 5/5
391/391 [==============================] - 38s 96ms/step - loss: 0.8721 - accuracy: 0.6984


In [79]:
quant_aware_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 1.1723 - accuracy: 0.6057


[1.1723408699035645, 0.6057000160217285]

In [118]:
cifar = tf.keras.datasets.cifar10.load_data()
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

In [119]:
_, baseline_model_accuracy = base_model.evaluate(
    x_test, y_test, verbose=2)

_, quant_aware_model_accuracy = quant_aware_model.evaluate(
   x_test, y_test, verbose=2)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quantize test accuracy:', quant_aware_model_accuracy)

313/313 - 14s - loss: 0.6396 - accuracy: 0.7966
313/313 - 3s - loss: 1.0360 - accuracy: 0.6530
Baseline test accuracy: 0.7965999841690063
Quantize test accuracy: 0.652999997138977


In [120]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)


converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model20 = converter.convert()

INFO:tensorflow:Assets written to: C:\Windows\TEMP\tmpi22v8y8q\assets


INFO:tensorflow:Assets written to: C:\Windows\TEMP\tmpi22v8y8q\assets


In [111]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [121]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model20)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', quant_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.1
Quant TF test accuracy: 0.652999997138977


In [122]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
float_tflite_model21 = float_converter.convert()


# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model20)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model21)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

INFO:tensorflow:Assets written to: C:\Windows\TEMP\tmppg8l6hua\assets


INFO:tensorflow:Assets written to: C:\Windows\TEMP\tmppg8l6hua\assets


Float model in Mb: 35.54994201660156
Quantized model in Mb: 1.4028549194335938
